# USA DATABASE UPLOAD

In [1]:
#REMEMBER TO DOUBLE CHECK IF THE WEEK OF THE PRICES QUERY AND FACT_TABLE QUERY MATCHES THE ONE YOURE CURRENTLY UPLOADING

#WALMART'S SoutMontoTrans DIFFERS BETWEEN BOTH METHODS IDK WHY, SINCE IT SHOULD BE THE SAME METHOD AS THE OTHER ONES.
#I'VE ALREADY DONE THE EXCERCISE MANUALLY AND THE RESULT IT'S THE SAME AS MINE, ALL THE OTHER COLUMNS ARE CORRECT

#AMAZON UNITS DON´T MATCH IF I DIVIDE BY THE UNIT PRICE, EVEN IF I ROUND THE NUMBERS UP OR DOWN

#NOT SURE HOW MIDWAY MAKES THE RELATIONSHIP BETWEEN THE CLIENTS AND OUR SucCodCliente, IT SEEMS THAT THEY ARE BEING GENERATED SEPARATELY

#AS OF RIGHT NOW, WALGREENS, TARGET AND WALMARTECOMMERCE ARE RUNNING PERFECTLY 

In [1]:
import warnings;
warnings.filterwarnings('ignore');

import jupyternotify
import pandas as pd
import os
import pyodbc

In [2]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

In [3]:
query_fact_table = '''
                    SELECT ProPstID
      ,SucID
      ,SoutCantDesp
      ,SoutCantExist
      ,SoutCantCedis
      ,SoutCantTrans
      ,SoutCantStaple
      ,SoutCantInv
      ,SoutMontoDesp
      ,SoutMontoExist
      ,SoutMontoCedis
      ,SoutMontoTrans
      ,SoutMontoStaple
      ,SoutMontoInv
      ,TmpID
      ,SoutMontoCteDesp
  FROM dbo.Fact_Desplazamiento_SemanalUSA
  where TmpID = '20210221'

'''

query_sucs = '''SELECT DISTINCT SucID, SucCodCliente, CadID
                FROM Gnm_MasterOp.dbo.vw_EstructuraSucursalesTotal
                WHERE SucID IS NOT NULL;'''

query_items='''
        SELECT 
        q.ProIDClie 
        ,p.CadID 
        ,p.ProPstID
        
        FROM dbo.OpHomologacionProdXCad as p, dbo.OpProductosHomologados as q
        where p.HpcID = q.HpcID 
'''

query_prices = '''
        SELECT *
  FROM dbo.GnmPreciosxCadenaHist where Año = 2021 and Semana = 37
'''

query_tmpid = '''
        SELECT TmpID
      ,TmpDiaSemana
      ,TmpSemanaAnioGenomma
      ,TmpAnioSemanaGenomma
      FROM dbo.Dim_Tiempo
'''

In [4]:
groups = {
    120:[110,314,315,343,344,345,346],
    145:[390,391,392,393,394,142],
    146:[143],
    302:[144],
    527:[766],
    730:[764],
    944:[1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1050,1051,1052,1053, 1077, 1078]
    }

df_tmpid = pd.read_sql(query_tmpid,conn1)
df_tmpid = df_tmpid[df_tmpid['TmpDiaSemana'] == 7]

In [5]:
def unique_prods(products):
    products = products[['ProIDClie','ProPstID']]
    products.drop_duplicates(inplace=True)
    prods_repetead = products['ProIDClie'].value_counts()[products['ProIDClie'].value_counts() > 1].index
    for prod in prods_repetead:
        min_ppst = products[products['ProIDClie'] == prod]['ProPstID'].max()
        products.loc[products['ProIDClie'] == prod, 'ProPstID'] = min_ppst
    products.drop_duplicates(inplace=True)
    products.reset_index(drop=True, inplace=True)
    return products

# Vermont

In [6]:
year = 2022
week = 9
week = str(week).zfill(2)

file = '{0}_{1}_VIP.xlsx'.format(year, week)
path = '../VIP Suerox/{0}/W {1} {0}/'.format(year, week)

df_vermont = pd.read_excel(path + file)
df_vermont[df_vermont.columns[-2]] = df_vermont[df_vermont.columns[-2]].astype(int) 
df_validation = df_vermont.copy()

df_vermont.columns = ['Distributors', 'Retail Accounts', 'Address', 'City', 'State',
       'Zip Code', 'VIP Outlet ID', 'UPC', 'Item Names',
       'Cases', 'Bottles',
       'USD']

df_vermont = df_vermont[['VIP Outlet ID', 'UPC', 'Bottles','USD']].copy()

df_vermont = pd.pivot_table(df_vermont,values=['Bottles','USD'],index=['VIP Outlet ID','UPC'], aggfunc='sum').reset_index()

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[944])]
df_items = unique_prods(df_items)

#STORES
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[944])]
df_stores_all = df_stores.copy()
df_stores.drop_duplicates('SucCodCliente',inplace=True, keep='last')
df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[944])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

#JOINS
df_vermont.rename({'VIP Outlet ID':'SucCodCliente'},axis=1,inplace=True)
df_vermont.rename({'UPC':'ProIDClie'},axis=1,inplace=True)
df_vermont['SucCodCliente'] = df_vermont['SucCodCliente'].astype(str)
df_vermont['ProIDClie'] = df_vermont['ProIDClie'].astype(str)

df_vermont = pd.merge(df_vermont,df_stores[['SucCodCliente','SucID','CadID']],on='SucCodCliente',how='left')
df_vermont = pd.merge(df_vermont,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
df_vermont['key'] = df_vermont['CadID'].astype(str) + '-' + df_vermont['ProPstID'].astype(str)
df_vermont = pd.merge(df_vermont,df_precios[['Precio Base','key']],on='key',how='left')

df_final = df_vermont[['ProPstID','SucID']].copy()

df_final['SoutCantDesp'] = df_vermont['Bottles']
df_final['SoutCantExist'] = 0
df_final['SoutCantCedis'] = 0
df_final['SoutCantTrans'] = 0
df_final['SoutCantStaple'] = 0
df_final['SoutCantInv'] = 0
df_final['SoutMontoDesp'] = df_vermont['Bottles'] * df_vermont['Precio Base']
df_final['SoutMontoExist'] = 0
df_final['SoutMontoCedis'] = 0
df_final['SoutMontoTrans'] = 0
df_final['SoutMontoStaple'] = 0
df_final['SoutMontoInv'] = 0
df_final['TmpID'] = df_tmpid.loc[(df_tmpid['TmpSemanaAnioGenomma'] == int(week)) & (df_tmpid['TmpAnioSemanaGenomma'] == year)]['TmpID'].unique()[0]
df_final['SoutMontoCteDesp'] = df_vermont['USD']

VERMONT = pd.pivot_table(df_final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist',
       'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv',
       'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans',
       'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

VERMONT = VERMONT[['ProPstID','SucID','SoutCantDesp','SoutCantExist','SoutCantCedis','SoutCantTrans','SoutCantStaple','SoutCantInv','SoutMontoDesp','SoutMontoExist','SoutMontoCedis','SoutMontoTrans','SoutMontoStaple','SoutMontoInv','TmpID','SoutMontoCteDesp']].copy()

x = df_validation.iloc[:,-2].sum()
y = VERMONT['SoutCantDesp'].sum()

print(f'Cifras Originales: {x}')
print(f'Cifras Finales: {y}')

Cifras Originales: 45395
Cifras Finales: 45395


**Checar las columnas *SoutMontoDesp* y *SoutMontoCteDesp*, ya que SoutMontoCteDesp está calculándolo bien.**

In [7]:
VERMONT['SoutMontoDesp'] = VERMONT['SoutMontoCteDesp']

In [8]:
VERMONT.isnull().sum()

ProPstID            0
SucID               0
SoutCantDesp        0
SoutCantExist       0
SoutCantCedis       0
SoutCantTrans       0
SoutCantStaple      0
SoutCantInv         0
SoutMontoDesp       0
SoutMontoExist      0
SoutMontoCedis      0
SoutMontoTrans      0
SoutMontoStaple     0
SoutMontoInv        0
TmpID               0
SoutMontoCteDesp    0
dtype: int64

In [9]:
VERMONT['SoutMontoDesp'].sum()

63815.70000000003

# Export Backup

In [10]:
path_output = '../VIP Suerox/Output/{0}/W {1} {0}'.format(year, week)

if not os.path.exists(path_output):
    os.mkdir(path_output)

VERMONT.to_csv(path_output + f'/VERMONT_W{week}.csv', index=False, encoding='latin1')

## Carga a DWH

In [11]:
os.listdir('../VIP Suerox/Output/2022')

['W 01 2022',
 'W 02 2022',
 'W 03 2022',
 'W 04 2022',
 'W 05 2022',
 'W 06 2022',
 'W 07 2022',
 'W 08 2022',
 'W 09 2022']

# Load data backup

In [12]:
year = 2022
weeks = [8, 9]
df_dwh = pd.DataFrame()

for week in weeks:
    week = str(week).zfill(2)
    path = '../VIP Suerox/Output/{0}/W {1} {0}/'.format(year, week)
    file = 'VERMONT_W{}.csv'.format(week)
    df_dwh = pd.concat([df_dwh,pd.read_csv(path + file)])

#Semanas a cargar
print(df_dwh['TmpID'].unique())
print(df_dwh.shape)

[20220220 20220227]
(140, 16)


# Load to SQL

In [13]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

cursor = conn.cursor()
# Carga a la tabla FillRate
## data es el df a cargar
for row in df_dwh.itertuples():
    cursor.execute('''
                INSERT INTO Gnm_DWH.dbo.Fact_Desplazamiento_SemanalUSA (ProPstID, SucID, SoutCantDesp, SoutCantExist, SoutCantCedis,SoutCantTrans, SoutCantStaple, SoutCantInv, SoutMontoDesp,SoutMontoExist, SoutMontoCedis, SoutMontoTrans, SoutMontoStaple,SoutMontoInv, TmpID, SoutMontoCteDesp)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''',
                row.ProPstID,
                row.SucID,
                row.SoutCantDesp,
                row.SoutCantExist,
                row.SoutCantCedis,
                row.SoutCantTrans,
                row.SoutCantStaple,
                row.SoutCantInv,
                row.SoutMontoDesp,
                row.SoutMontoExist,
                row.SoutMontoCedis,
                row.SoutMontoTrans,
                row.SoutMontoStaple,
                row.SoutMontoInv,
                row.TmpID,
                row.SoutMontoCteDesp
                )
conn.commit()
print(str(df_dwh.shape[0]) + " registros cargados")
cursor.close()

140 registros cargados


## TARGET

In [8]:
#DATA
file = '2021_10_Target'
df_target = pd.read_excel('Datos/'+ file +'.xlsx')
df_target = df_target[['DPCI','LOCATION NUMBER', 'LOCATION','REG SALES U']]
df_target.dropna(how='all',inplace=True)
df_target.reset_index(drop=True, inplace=True)
df_target['DPCI'] = df_target['DPCI'].apply(lambda x: x.replace('-',''))
week = int(file.split('_')[1])
year = int(file.split('_')[0])

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[145])]
df_items = unique_prods(df_items)

#STORES
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[145])]
df_stores_all = df_stores.copy()
df_stores.drop_duplicates('SucCodCliente',inplace=True, keep='last')
df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[145])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

#JOINS
df_target.rename({'LOCATION NUMBER':'SucCodCliente'},axis=1,inplace=True)
df_target.rename({'DPCI':'ProIDClie'},axis=1,inplace=True)
df_target['SucCodCliente'] = df_target['SucCodCliente'].astype(str)
df_target['ProIDClie'] = df_target['ProIDClie'].astype(str)

df_target = pd.merge(df_target,df_stores[['SucCodCliente','SucID','CadID']],on='SucCodCliente',how='left')
df_target = pd.merge(df_target,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
df_target['key'] = df_target['CadID'].astype(str) + '-' + df_target['ProPstID'].astype(str)
df_target = pd.merge(df_target,df_precios[['Precio Base','key']],on='key',how='left')

df_final = df_target[['ProPstID','SucID']].copy()

df_final['SoutCantDesp'] = df_target['REG SALES U']
df_final['SoutCantExist'] = 0
df_final['SoutCantCedis'] = 0
df_final['SoutCantTrans'] = 0
df_final['SoutCantStaple'] = 0
df_final['SoutCantInv'] = 0
df_final['SoutMontoDesp'] = df_target['REG SALES U'] * df_target['Precio Base']
df_final['SoutMontoExist'] = 0
df_final['SoutMontoCedis'] = 0
df_final['SoutMontoTrans'] = 0
df_final['SoutMontoStaple'] = 0
df_final['SoutMontoInv'] = 0
df_final['TmpID'] = df_tmpid.loc[(df_tmpid['TmpSemanaAnioGenomma'] == week) & (df_tmpid['TmpAnioSemanaGenomma'] == year)]['TmpID'].unique()[0]
df_final['SoutMontoCteDesp'] = 0

TARGET = pd.pivot_table(df_final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist',
       'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv',
       'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans',
       'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

TARGET = TARGET[['ProPstID','SucID','SoutCantDesp','SoutCantExist','SoutCantCedis','SoutCantTrans','SoutCantStaple','SoutCantInv','SoutMontoDesp','SoutMontoExist','SoutMontoCedis','SoutMontoTrans','SoutMontoStaple','SoutMontoInv','TmpID','SoutMontoCteDesp']]

In [9]:
df_target[df_target['SucID'].isnull()]

,ProIDClie,SucCodCliente,LOCATION,REG SALES U,SucID,CadID,ProPstID,key,Precio Base


In [11]:
TARGET.to_csv('target_w10.csv', index=False, encoding='latin1')

## MIDWAY

In [121]:
#DATA
file = '2021_02_Midway'
df_midway = pd.read_excel('Datos/'+ file +'.xls',usecols='I,K,M,Q,S',header=1)
df_midway.dropna(how='all',inplace=True)
df_midway.reset_index(drop=True,inplace=True)
df_midway['ItemID'] = df_midway['ItemID'].apply(lambda x: x.replace('-',''))
week = int(file.split('_')[1])
year = int(file.split('_')[0])

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[120])]
df_items = unique_prods(df_items)

#STORES
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[120])]
df_stores.drop_duplicates('SucCodCliente',inplace=True)
df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[120])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

#JOINS
# df_target.rename({'LOCATION NUMBER':'SucCodCliente'},axis=1,inplace=True)
# df_target.rename({'DPCI':'ProIDClie'},axis=1,inplace=True)
# df_target['SucCodCliente'] = df_target['SucCodCliente'].astype(str)
# df_target['ProIDClie'] = df_target['ProIDClie'].astype(str)

# df_target = pd.merge(df_target,df_stores[['SucCodCliente','SucID','CadID']],on='SucCodCliente',how='left')
# df_target = pd.merge(df_target,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
# df_target['key'] = df_target['CadID'].astype(str) + '-' + df_target['ProPstID'].astype(str)
# df_target = pd.merge(df_target,df_precios[['Precio Base','key']],on='key',how='left')




<ipython-input-5-121899746c0c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.drop_duplicates(inplace=True)
C:\Users\jvelazquezh\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-5-121899746c0c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.drop_duplicates(inplace=True)


## Walmart

In [8]:
#3 Dolares de diferencia en soutmontotrans
week = 10
year = 2021

#DATA
BTY =  pd.read_excel('Datos/Walmart/W 10 2021/Beauty 03.08.2021.xlsx',header = 20)
OTC1 = pd.read_excel('Datos/Walmart/W 10 2021/OTC 1 03.08.2021.xlsx',header = 20)
OTC2 = pd.read_excel('Datos/Walmart/W 10 2021/OTC 2 03.08.2021.xlsx',header = 20)
PR = pd.read_excel('Datos/Walmart/W 10 2021/Copy of Genomma Labs Week 202105.xlsx',header=18)
df_walmart = pd.concat([BTY,OTC1,OTC2,PR])
df_walmart.reset_index(drop=True,inplace=True)
df_walmart.loc[df_walmart['UPC'] == 65024001848, 'UPC'] = 65006700046
df_walmart.loc[df_walmart['UPC'] == 65024001847, 'UPC'] = 65006600048

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[302])]
df_items = unique_prods(df_items)

#STORES
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[302])]
df_stores_all = df_stores.copy()
df_stores.drop_duplicates('SucCodCliente',inplace=True)
df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[302])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

#JOINS
df_walmart.rename({'Store Nbr':'SucCodCliente'},axis=1,inplace=True)
df_walmart.rename({'UPC':'ProIDClie'},axis=1,inplace=True)
df_walmart['SucCodCliente'] = df_walmart['SucCodCliente'].astype(str)
df_walmart['ProIDClie'] = df_walmart['ProIDClie'].astype(str)

df_walmart = pd.merge(df_walmart,df_stores[['SucCodCliente','SucID','CadID']],on='SucCodCliente',how='left')
df_walmart = pd.merge(df_walmart,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
#df_walmart['key'] = df_walmart['CadID'].astype(str) + '-' + df_walmart['ProPstID'].astype(str)
df_walmart = pd.merge(df_walmart,df_precios[['Precio Base','ProPstID']],on='ProPstID',how='left') #Quite la llave de cadena para walmart

df_final = df_walmart[['ProPstID','SucID']].copy()

df_final['SoutCantDesp'] = df_walmart['POS Qty']
df_final['SoutCantExist'] = df_walmart['Curr Str On Hand Qty']
df_final['SoutCantCedis'] = 0
df_final['SoutCantTrans'] = df_walmart['Curr Str In Transit Qty']
df_final['SoutCantStaple'] = 0
df_final['SoutCantInv'] = df_final['SoutCantExist'] + df_walmart['Curr Str In Transit Qty']
df_final['SoutMontoDesp'] = df_walmart['POS Qty'] * df_walmart['Precio Base']
df_final['SoutMontoExist'] = df_walmart['Curr Str On Hand Qty'] * df_walmart['Precio Base']
df_final['SoutMontoCedis'] = 0
df_final['SoutMontoTrans'] = df_walmart['Curr Str In Transit Qty'] * df_walmart['Precio Base']
df_final['SoutMontoStaple'] = 0
df_final['SoutMontoInv'] = df_final['SoutMontoExist'] + df_final['SoutMontoTrans']
df_final['TmpID'] = df_tmpid.loc[(df_tmpid['TmpSemanaAnioGenomma'] == week) & (df_tmpid['TmpAnioSemanaGenomma'] == year)]['TmpID'].unique()[0]
df_final['SoutMontoCteDesp'] = df_walmart['POS Sales']

WALMART = pd.pivot_table(df_final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist',
       'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv',
       'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans',
       'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

WALMART = WALMART[['ProPstID','SucID','SoutCantDesp','SoutCantExist','SoutCantCedis','SoutCantTrans','SoutCantStaple','SoutCantInv','SoutMontoDesp','SoutMontoExist','SoutMontoCedis','SoutMontoTrans','SoutMontoStaple','SoutMontoInv','TmpID','SoutMontoCteDesp']]

In [9]:
df_walmart[df_walmart['SucID'].isnull()][['SucCodCliente','Street Address','City','State','Zip Code']]

,SucCodCliente,Street Address,City,State,Zip Code


In [10]:
df_walmart[df_walmart['ProPstID'].isnull()][['ProIDClie','Item Desc 1']].drop_duplicates()

,ProIDClie,Item Desc 1


In [11]:
WALMART.to_csv('walmart_w10.csv', index=False, encoding='latin1')

## WALGREENS

In [18]:
#Aqui no se hace la relacion por el key, falta homologar el codigo a como los anteriores
#FILES
file = '2021_10_Walgreens_SO'
df_walgreens = pd.read_excel('Datos/'+ file +'.xlsx')
df_walgreens.rename({'STORE':'SucCodCliente'},axis=1,inplace=True)
df_walgreens.rename({'UPC':'ProIDClie'},axis=1,inplace=True)
df_walgreens['SucCodCliente'] = df_walgreens['SucCodCliente'].astype(str)
df_walgreens['ProIDClie'] = df_walgreens['ProIDClie'].astype(str)
week = int(file.split('_')[1])
year = int(file.split('_')[0])

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[146])]
df_items = unique_prods(df_items)

#STORES
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[146])]
df_stores_all = df_stores.copy()
df_stores.drop_duplicates('SucCodCliente',inplace=True)
df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[146])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

df_walgreens = pd.merge(df_walgreens,df_stores[['SucCodCliente','SucID','CadID']],on='SucCodCliente',how='left')
df_walgreens = pd.merge(df_walgreens,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
df_walgreens = pd.merge(df_walgreens,df_precios[['ProPstID','Precio Base']],on='ProPstID',how='left')

df_final = df_walgreens[['ProPstID','SucID']].copy()

df_final['SoutCantDesp'] = df_walgreens['Total Sales Volume Units']
df_final['SoutCantExist'] = df_walgreens['Store On Hand Volume Units']
df_final['SoutCantCedis'] = 0
df_final['SoutCantTrans'] = 0
df_final['SoutCantStaple'] = 0
df_final['SoutCantInv'] = df_final['SoutCantExist']
df_final['SoutMontoDesp'] = df_walgreens['Total Sales Volume Units'] * df_walgreens['Precio Base']
df_final['SoutMontoExist'] = df_walgreens['Store On Hand Volume Units'] * df_walgreens['Precio Base']
df_final['SoutMontoCedis'] = 0
df_final['SoutMontoTrans'] = 0
df_final['SoutMontoStaple'] = 0
df_final['SoutMontoInv'] = df_walgreens['Store On Hand Volume Units'] * df_walgreens['Precio Base']
df_final['TmpID'] = df_tmpid.loc[(df_tmpid['TmpSemanaAnioGenomma'] == week) & (df_tmpid['TmpAnioSemanaGenomma'] == year)]['TmpID'].unique()[0]
df_final['SoutMontoCteDesp'] = df_walgreens['Total Sales Amount']

WALGREENS = pd.pivot_table(df_final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist',
       'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv',
       'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans',
       'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

WALGREENS = WALGREENS[['ProPstID','SucID','SoutCantDesp','SoutCantExist','SoutCantCedis','SoutCantTrans','SoutCantStaple','SoutCantInv','SoutMontoDesp','SoutMontoExist','SoutMontoCedis','SoutMontoTrans','SoutMontoStaple','SoutMontoInv','TmpID','SoutMontoCteDesp']]

In [19]:
df_walgreens[df_walgreens['SucID'].isnull()][['SucCodCliente','ADDRESS1','CITY','STATE','ZIP']].drop_duplicates('SucCodCliente')

,SucCodCliente,ADDRESS1,CITY,STATE,ZIP


In [20]:
df_walgreens.isnull().sum()

ProIDClie                     0
ITEM DESCRIPTION              0
ADDRESS1                      0
CITY                          0
STATE                         0
ZIP                           0
SucCodCliente                 0
Total Sales Amount            0
Total Sales Volume Units      0
Store On Hand Volume Units    0
SucID                         0
CadID                         0
ProPstID                      0
Precio Base                   0
dtype: int64

In [21]:
WALGREENS.to_csv('walgreens_w10.csv', index=False, encoding='latin1')

## WALMART ECOMMERCE

In [12]:
#FILES
df_walmarte = pd.read_excel('Datos/Walmart/W 10 2021/2021_10_WalmarteCommerce.xlsx',header = 11)
df_walmarte.loc[df_walmarte['UPC'] == 65024001848, 'UPC'] = 65006700046
df_walmarte.loc[df_walmarte['UPC'] == 65024001847, 'UPC'] = 65006600048
df_walmarte['UPC'] = '00' + df_walmarte['UPC'].astype(str) #Los codigos para eCommerce son 00 + upc
week = int(file.split('_')[1])
year = int(file.split('_')[0])

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[730])]
df_items = unique_prods(df_items)

#STORES
# df_stores = pd.read_sql(query_sucs,conn2)
# df_stores = df_stores[df_stores.CadID.isin(groups[730])]
# df_stores.drop_duplicates('SucCodCliente',inplace=True)
# df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[730])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

#JOINS
df_walmarte.rename({'UPC':'ProIDClie'},axis=1,inplace=True)
df_walmarte['ProIDClie'] = df_walmarte['ProIDClie'].astype(str)

df_walmarte['SucID'] = 139013
df_walmarte['CadID'] = 764
df_walmarte = pd.merge(df_walmarte,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
df_walmarte['key'] = df_walmarte['CadID'].astype(str) + '-' + df_walmarte['ProPstID'].astype(str)
df_walmarte = pd.merge(df_walmarte,df_precios[['Precio Base','key']],on='key',how='left')

df_final = df_walmarte[['ProPstID','SucID']].copy()

df_final['SoutCantDesp'] = df_walmarte['Total Net Unit Sales']
df_final['SoutCantExist'] = 0
df_final['SoutCantCedis'] = 0
df_final['SoutCantTrans'] = 0
df_final['SoutCantStaple'] = 0
df_final['SoutCantInv'] = 0
df_final['SoutMontoDesp'] = df_walmarte['Total Net Unit Sales'] * df_walmarte['Precio Base']
df_final['SoutMontoExist'] = 0
df_final['SoutMontoCedis'] = 0
df_final['SoutMontoTrans'] = 0
df_final['SoutMontoStaple'] = 0
df_final['SoutMontoInv'] = 0
df_final['TmpID'] = df_tmpid.loc[(df_tmpid['TmpSemanaAnioGenomma'] == week) & (df_tmpid['TmpAnioSemanaGenomma'] == year)]['TmpID'].unique()[0]
df_final['SoutMontoCteDesp'] = df_walmarte['Total Net Retail Sales']

df_final.loc[df_final['SoutCantDesp']<0,'SoutMontoCteDesp'] = 0

WALMARTECOMMERCE = pd.pivot_table(df_final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist',
       'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv',
       'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans',
       'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

WALMARTECOMMERCE = WALMARTECOMMERCE[['ProPstID','SucID','SoutCantDesp','SoutCantExist','SoutCantCedis','SoutCantTrans','SoutCantStaple','SoutCantInv','SoutMontoDesp','SoutMontoExist','SoutMontoCedis','SoutMontoTrans','SoutMontoStaple','SoutMontoInv','TmpID','SoutMontoCteDesp']]

In [13]:
df_walmarte[df_walmarte['ProPstID'].isnull()][['ProIDClie','Item Name']].drop_duplicates()

,ProIDClie,Item Name


In [14]:
WALMARTECOMMERCE['SoutCantDesp'].sum()

184

In [15]:
WALMARTECOMMERCE.to_csv('walmartecommerce_w10.csv', index=False, encoding='latin1')

## AMAZON

In [52]:
df_amazon_original = pd.read_excel('Datos/2021_08_Amazon.xlsx',header = 1)
df_amazon_original = df_amazon_original.iloc[:-1,:]
df_amazon = df_amazon_original[['Product Title','UPC','List Price to Calculate Units','Brand']]
df_amazon['UPC'] = df_amazon['UPC'].astype('int64') 
df_amazon['Sales'] = df_amazon_original.iloc[:,-1:]
df_amazon['Units'] = df_amazon['Sales']/df_amazon['List Price to Calculate Units']
#df_amazon['Units'] = df_amazon['Units'].apply(lambda x: math.floor(x))
week = int(file.split('_')[1])
year = int(file.split('_')[0])

#ITEMS
df_items = pd.read_sql(query_items,conn2)
df_items = df_items[df_items.CadID.isin(groups[527])]
df_items = unique_prods(df_items)

#STORES
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[527])]
df_stores.drop_duplicates('SucCodCliente',inplace=True)
df_stores.reset_index(drop=True,inplace=True)

#PRICES
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[527])]
df_precios.reset_index(drop=True,inplace=True)
df_precios['key'] = df_precios['CadID'].astype(str) + '-' + df_precios['ProPstID'].astype(str)

#JOINS
df_amazon.rename({'UPC':'ProIDClie'},axis=1,inplace=True)
df_amazon['ProIDClie'] = df_amazon['ProIDClie'].astype(str)

df_amazon['SucID'] = 137408
df_amazon['CadID'] = 766
df_amazon = pd.merge(df_amazon,df_items[['ProIDClie','ProPstID']],on='ProIDClie',how='left')
df_amazon['key'] = df_amazon['CadID'].astype(str) + '-' + df_amazon['ProPstID'].astype(str)
df_amazon = pd.merge(df_amazon,df_precios[['Precio Base','key']],on='key',how='left')
df_amazon['SoutMontoDesp'] = df_amazon['Precio Base'] * df_amazon['Units']

df_final = df_amazon[['SucID','ProPstID']].copy()

df_final['SoutCantDesp'] = df_amazon['Units']
df_final['SoutCantExist'] = 0
df_final['SoutCantCedis'] = 0
df_final['SoutCantTrans'] = 0
df_final['SoutCantStaple'] = 0
df_final['SoutCantInv'] = 0
df_final['SoutMontoDesp'] = df_amazon['SoutMontoDesp']
df_final['SoutMontoExist'] = 0
df_final['SoutMontoCedis'] = 0
df_final['SoutMontoTrans'] = 0
df_final['SoutMontoStaple'] = 0
df_final['SoutMontoInv'] = 0
df_final['TmpID'] = df_tmpid.loc[(df_tmpid['TmpSemanaAnioGenomma'] == week) & (df_tmpid['TmpAnioSemanaGenomma'] == year)]['TmpID'].unique()[0]
df_final['SoutMontoCteDesp'] = df_amazon['Sales']

AMAZON = pd.pivot_table(df_final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist',
       'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv',
       'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans',
       'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

AMAZON = AMAZON[['ProPstID','SucID','SoutCantDesp','SoutCantExist','SoutCantCedis','SoutCantTrans','SoutCantStaple','SoutCantInv','SoutMontoDesp','SoutMontoExist','SoutMontoCedis','SoutMontoTrans','SoutMontoStaple','SoutMontoInv','TmpID','SoutMontoCteDesp']]

DatabaseError: Execution failed on sql: 
        SELECT 
        
        q.ProIDClie 
        ,p.CadID 
        ,p.ProPstID
        
        FROM dbo.OpHomologacionProdXCad as p, dbo.OpProductosHomologados as q
        where p.HpcID = q.HpcID 

('08S01', '[08S01] [Microsoft][ODBC SQL Server Driver]Error en el vínculo de comunicación (0) (SQLExecDirectW)')
unable to rollback

In [42]:
AMAZON['SoutCantDesp'].sum()

7536

In [43]:
df_amazon.to_csv('AMAZON.csv',encoding='latin1')

In [47]:
df_amazon.loc[21,:]

Product Title                    CICATRICURE Vitamin C Serum, 1 Ounce
ProIDClie                                                650240055010
List Price to Calculate Units                                     9.3
Brand                                                     CICATRICURE
Sales                                                           102.3
Units                                                              10
SucID                                                          137408
CadID                                                             766
ProPstID                                                         8897
key                                                          766-8897
Precio Base                                                       9.3
SoutMontoDesp                                                      93
Name: 21, dtype: object

## REVIEW VS DATABASE

In [36]:
x = WALGREENS
for col in x.columns:
    print(col)
    print(x[col].sum())

ProPstID
2970604006
SucID
14363599728
SoutCantDesp
62655
SoutCantExist
1142662
SoutCantCedis
0
SoutCantTrans
0
SoutCantStaple
0
SoutCantInv
1142662
SoutMontoDesp
374873.94
SoutMontoExist
7379732.689999999
SoutMontoCedis
0
SoutMontoTrans
0
SoutMontoStaple
0
SoutMontoInv
7379732.689999999
TmpID
12043208720300
SoutMontoCteDesp
659487.8100000002


In [35]:
x = df_USA[df_USA.SucID.isin(list(df_stores_all['SucID']))]
#x = df_USA[df_USA.SucID.isin([139013])]
for col in x.columns:
    print(col)
    print(x[col].sum())

ProPstID
2954761242
SucID
14348628472
SoutCantDesp
62436
SoutCantExist
1142662
SoutCantCedis
0
SoutCantTrans
0
SoutCantStaple
0
SoutCantInv
1142662
SoutMontoDesp
373670.1600000001
SoutMontoExist
7379732.689999999
SoutMontoCedis
0.0
SoutMontoTrans
0.0
SoutMontoStaple
0.0
SoutMontoInv
7379732.689999999
TmpID
12033669545076
SoutMontoCteDesp
657433.6499999999


In [54]:
df_stores = pd.read_sql(query_sucs,conn2)
df_stores = df_stores[df_stores.CadID.isin(groups[145])]

In [62]:
df_stores[df_stores.duplicated('SucCodCliente',keep='last')]

,SucID,SucCodCliente,CadID
134400,119256,T0055,142
135249,119257,T1394,142


In [7]:
%%time
df_USA = pd.read_sql(query_fact_table,conn1)

Wall time: 1min 37s


In [10]:
df_USA.to_csv('UnaSemana.csv',encoding='latin1',index=False)

In [8]:
import sys

print(sys.getsizeof(df_USA))

87957264


In [34]:
x = df[df.SucID.isin(list(join['SucID'].unique()))]
for col in x.columns:
    print(col)
    print(x[col].sum())

ProPstID
2953886786
SucID
14342044830
SoutCantDesp
69870
SoutCantExist
1160699
SoutCantCedis
0
SoutCantTrans
0
SoutCantStaple
0
SoutCantInv
1160699
SoutMontoDesp
420161.1400000001
SoutMontoExist
7493857.82
SoutMontoCedis
0.0
SoutMontoTrans
0.0
SoutMontoStaple
0.0
SoutMontoInv
7493857.82
TmpID
12030795541460
SoutMontoCteDesp
732984.3599999999


In [114]:
data_final2.to_excel('Walgreens_week3.xlsx')

In [29]:
df_precios = pd.read_sql(query_prices,conn2)
df_precios = df_precios[df_precios.CadID.isin(groups[302])]

In [33]:
df_precios['ProPstID'].value_counts()

4152    1
2379    1
2392    1
2390    1
2389    1
       ..
4246    1
4501    1
4500    1
4499    1
4352    1
Name: ProPstID, Length: 184, dtype: int64

In [62]:
df_items.dtypes

HpcID         int64
ProIDClie    object
CadID         int64
ProPstID      int64
dtype: object

In [55]:
df_items

,HpcID,ProIDClie,CadID,ProPstID
0,37314,65006600065,143,3981
1,37315,65024002985,143,3982
2,37316,65006600064,143,3983
3,37277,65024002980,143,3959
4,43319,65024001642,143,4772
...,...,...,...,...
235,48963,65024001830,143,5413
236,58473,65006600202,143,7409
237,57365,65006600226,143,6198
238,39536,65024003173,143,4287


In [36]:
df_walgreens['Total Sales Amount'].sum()

732984.3599999998

In [29]:
t = list(df_stores_all['SucID'].astype(str))

In [30]:
",".join(t)

'56145,49683,73435,8555,46448,18129,18552,28833,28834,18720,137458,18239,24292,20615,20616,28133,8424,17946,18130,8538,25623,8618,17495,16531,27693,16613,20617,10684,10783,18319,20618,20619,20620,20621,20622,16500,20623,20624,20625,20626,20627,14382,16614,18298,8475,16501,16615,28126,17285,16457,20628,8463,17239,18721,20629,20630,15933,16616,20631,20632,8541,20633,15948,8059,16617,8328,8056,18512,8302,10569,11228,20634,10888,20635,20636,12678,7907,16046,7965,20637,20638,7552,20639,20640,20641,18201,20642,21467,20643,16618,10627,20644,20645,20646,16321,20647,20648,16619,20649,20650,17945,18202,20651,8253,20652,20653,20654,20655,20656,20657,16620,39810,16621,16622,20658,16623,20659,20660,20661,8627,10542,16322,20662,20663,8547,20664,20665,20666,25922,20667,8238,20668,20669,20670,20671,8154,20672,20673,21468,11404,8584,8286,10592,20674,20675,20676,18688,16217,20677,20678,20679,18528,23740,17837,21469,20680,20681,20682,8491,17265,20683,20684,20685,18347,20686,11663,18306,16624,12674,20687,

In [41]:
WEEK_3 = pd.concat([df_USA[~df_USA.SucID.isin(list(df_stores_all['SucID']))],WALGREENS],axis=0)
WEEK_3.reset_index(drop=True,inplace=True)

In [43]:
WEEK_3[WEEK_3.SucID.isin(list(df_stores_all['SucID']))]['SoutCantDesp'].sum()

62655

In [44]:
WEEK_3.to_csv('20210117.csv',index=False,encoding='latin1')